## 바텀 라인의 영향력 분석

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from urllib.parse import quote as quotekr # 한글 아이디를 인코딩 parse.quote
import json
from datetime import datetime
import time

In [2]:
def printjs(result):
    print(json.dumps(result, ensure_ascii=False, indent=2))

In [3]:
api_key = 'RGAPI-d756acf2-4934-493b-a2d4-3d4a412dd920'
request_header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
                "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
                "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                "Origin": "https://developer.riotgames.com",
                "X-Riot-Token": api_key
                }

#### 티어별 유저 5명 선별
* op.gg에서 무작위 추출
* Most top 3 챔피언이 모두 원거리 딜러
* 랭크 선호 포지션이 바텀 비율이 월등히 높은 사람
* 솔로랭크 50판 이상

In [7]:
grand_master_user = ['줄라이77', '바텀개', 'always crying', '침대위섹시카이팅', '미드KING']
diamond_user = ['세혀닝', '강한남자한만윤', 'FEARN0T', '잘해봐요괜찮아요', '쿠로미짱이현']
emerald_user = ['정의는 승리한다v', 'no return0', 'Cigar3', '주희보고싶은민교', 'Asnm']
platinum_user = ['협곡의 맙소사', '엄딴식', '지구뚝배기', '08년생 큐티정민', 'Gucci Vs']
gold_user = ['루픠는못말려', '홍구형', '카이팅의 귀재', 'Orangerie', '쥐파티']

grand_master_puuid = []
diamond_puuid = []
emerald_puuid = []
platinum_puuid = []
gold_puuid = []

In [5]:
def user_puuid(users, puuids):
    for user in users:
        url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
        response = requests.get(url, headers=request_header)
        
        if response.status_code == 200: # Success
            pass

        elif response.status_code == 429: # Rate limit exceeded
            print('Rate limit exceeded error, api cost full')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 5 second wait time')
                    time.sleep(5)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 503: # Service unavailable
            print('Service unavailable error')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 5 second wait time')
                    time.sleep(5)
                    response = requests.get(url, headers=request_header)

        elif req.status_code == 403: # Forbidden
            print('Forbidden error, you need api renewal')
            print('break')
            return

        puuids.append(response.json()['puuid'])

In [8]:
user_puuid(grand_master_user, grand_master_puuid)
user_puuid(diamond_user, diamond_puuid)
user_puuid(emerald_user, emerald_puuid)
user_puuid(platinum_user, platinum_puuid)
user_puuid(gold_user, gold_puuid)

In [12]:
gold_puuid

['Kf7RUV7nOSiqQ_jHKIrhcUWLUxt-XNseED9aEl5LDV1APTMUXoPdIwuvEEAHUxOb3ckHX3k6tAfl9Q',
 '6ACA-XanXWYRdej-omBrnlM-ziv_txp6U-5oKmtLr0yqhutM16Hcb_NUqHwhszsTAjST2MD30XTHiQ',
 'NpPspwOHeTBXQLvGH3q-1w9cOqy5D12CK50YMppRW_2GlGEI2uRhyTiVTS-9F7cGCKhhqGGdhnf1jw',
 'lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGMUGoukukJFCBJazgNocw8Wh2XSAKfhZ4A',
 'CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw6yQzHdLJ8f6CAEN6C2YHGJZyRYoBma0Q']

In [21]:
grand_master_df = pd.DataFrame({
    'tier': ['grand_master' for i in range(5)],
    'summoner': grand_master_user,
    'puuid': grand_master_puuid
    })

diamond_df = pd.DataFrame({
    'tier': ['diamond' for i in range(5)],
    'summoner': diamond_user,
    'puuid': diamond_puuid
    })

emerald_df = pd.DataFrame({
    'tier': ['emerald' for i in range(5)],
    'summoner': emerald_user,
    'puuid': emerald_puuid
    })

platinum_df = pd.DataFrame({
    'tier': ['platinum' for i in range(5)],
    'summoner': platinum_user,
    'puuid': platinum_puuid
    })

gold_df = pd.DataFrame({
    'tier': ['gold' for i in range(5)],
    'summoner': gold_user,
    'puuid': gold_puuid
    })

In [24]:
df = pd.concat([grand_master_df, diamond_df, emerald_df, platinum_df, gold_df], ignore_index=True)
display(df.head(2))
display(df.tail(2))

,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


,tier,summoner,puuid
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...


#### 데이터 수집
* '바텀 게임' 이라는 말이 줄기 시작한 13.6패치가 3월 중순
* 1주일에 랭크게임 5판 한다고 가정
* 현재 8월 말이므로 약 20주
* 100게임 이전 인당 10판

In [38]:
# 랭크 게임 : len(response.json()['info']['teams'][0]['bans']) == 0 (자유랭크도 포함됨)

def puuid_to_matchid(puuid):
    match_ids = []
    start = 100

    while len(match_ids) < 10:
        matches_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count=20'
        match_list_resp = requests.get(matches_url, headers=request_header)
        if match_list_resp.status_code == 200:
            match_list = match_list_resp.json()

            for mat_id in match_list:
                if len(match_ids) == 10:
                    break

                mat_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mat_id
                mat_data_resp = requests.get(mat_url, headers=request_header)
                if mat_data_resp.status_code == 200:
                    mat_data = mat_data_resp.json()
                    if len(mat_data['info']['teams'][0]['bans']) > 0: # 밴이 있는 경기 -> 랭크 게임
                        match_ids.append(mat_id)

                else:
                    print("\tmatch data response error")
                    return
                
            start += 20

    return match_ids


In [61]:
def puuid_to_matchlist(puuid):
    matches_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=100&count=100'
    match_list_resp = requests.get(matches_url, headers=request_header)

    if match_list_resp.status_code == 200:
        match_list = match_list_resp.json()
        return match_list
    else:
        print("error 1")
        return []

In [62]:
def matchlist_to_ids(match_list):
    if len(match_list) == 0:
        print("error 2")
        return

    match_ids = []
    for mat_id in match_list:
        if len(match_ids) == 10:
            break

        mat_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{mat_id}'
        mat_data_resp = requests.get(mat_url, headers=request_header)
        if mat_data_resp.status_code == 200:
            mat_data = mat_data_resp.json()
            if len(mat_data['info']['teams'][0]['bans']) > 0: # 밴이 있는 경기 -> 랭크 게임
                match_ids.append(mat_id)

        else:
            print("error 3")
            return []
            
    return match_ids


In [47]:
col = [('match' + str(x)) for x in range(1, 11)]
print(col)

['match1', 'match2', 'match3', 'match4', 'match5', 'match6', 'match7', 'match8', 'match9', 'match10']


In [65]:
df_puuid = pd.DataFrame(columns=col)

In [ ]:
pu_ml = puuid_to_matchlist(df['puuid'][24])
ml_id = matchlist_to_ids(pu_ml)
print(ml_id)

df_rank_ids = pd.DataFrame([ml_id[::-1]], columns=col)
df_puuid = pd.concat([df_puuid, df_rank_ids], ignore_index=True)
display(df_puuid)

In [102]:
df2 = pd.concat([df, df_puuid], axis=1)
df2

,tier,summoner,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6626090481,KR_6626176054,KR_6626235450,KR_6635642133,KR_6635646669,KR_6635664272,KR_6635676154,KR_6635779706,KR_6635823372,KR_6635868924
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...,KR_6655383970,KR_6656228346,KR_6656245832,KR_6656268203,KR_6656270914,KR_6656299201,KR_6656316608,KR_6656343443,KR_6656377096,KR_6656390997
2,grand_master,always crying,wBQaUzCDHu1SyLD330DHqGeJeFo3PhDeaYeyjlgqyp7SG0...,KR_6651583984,KR_6651641733,KR_6651718581,KR_6656479499,KR_6656523375,KR_6656551553,KR_6657320045,KR_6657512458,KR_6657626230,KR_6658072186
3,grand_master,침대위섹시카이팅,4ZSpLbeybSbDG-s52ZQSsxQdGvtQbKZdvf_B-UwZ3P1_DJ...,KR_6597517687,KR_6597533457,KR_6597545311,KR_6599491392,KR_6599528440,KR_6599565457,KR_6601445620,KR_6616309182,KR_6616365537,KR_6641362367
4,grand_master,미드KING,xr50XOMRd-wO_P8iKDaPiz1G45o15o8atDxQq8LDqrLxQY...,KR_6597383901,KR_6597423923,KR_6597458443,KR_6600350874,KR_6600518396,KR_6600652224,KR_6600673405,KR_6600690424,KR_6603518349,KR_6603571667
5,diamond,세혀닝,Q5YNI7bq1mrMB00HaWSomEW-A52vhXj9THL3pFS6b4sxQn...,KR_6664985797,KR_6665017736,KR_6665060147,KR_6665073852,KR_6665116347,KR_6665137816,KR_6665165870,KR_6665199915,KR_6665567475,KR_6665627855
6,diamond,강한남자한만윤,fybIOHNUpFHR4TcPoC8vyhKxVrYsZqV9ornEHNqL3ajkIn...,KR_6566183643,KR_6568637327,KR_6568744453,KR_6572437459,KR_6575672546,KR_6575757937,KR_6575846692,KR_6579050872,KR_6579125571,KR_6579173068
7,diamond,FEARN0T,oZ4b0WJY2fP71MYT8BOpzUBSxUHG7pBloo-aGMWYzS09Q3...,KR_6574611605,KR_6598850042,KR_6598901712,KR_6604637117,KR_6604700058,KR_6607917861,KR_6608049851,KR_6608149766,KR_6608762738,KR_6608803664
8,diamond,잘해봐요괜찮아요,2r4JbtA47RsnOICJJ1VjKO8-ogfvy5QytzAHTmKOyolaLq...,KR_6643645803,KR_6644053216,KR_6644176053,KR_6644273750,KR_6645628791,KR_6645672689,KR_6645738229,KR_6645834148,KR_6645902831,KR_6646001285
9,diamond,쿠로미짱이현,c3jkQfup6vcyiaLtTT0a5IZlHgiF40sq0v1MvGE2Pm6hib...,KR_6611273171,KR_6611286387,KR_6611298094,KR_6611293739,KR_6613147937,KR_6613182086,KR_6613189683,KR_6613199629,KR_6613216264,KR_6613225411


In [12]:
df2.to_excel('matchid_data.xlsx', index=False)

In [4]:
df2 = pd.read_excel('matchid_data.xlsx')

In [135]:
df2.loc[[0]]

,tier,summoner,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6626090481,KR_6626176054,KR_6626235450,KR_6635642133,KR_6635646669,KR_6635664272,KR_6635676154,KR_6635779706,KR_6635823372,KR_6635868924


#### 경기 데이터 전처리

In [42]:
match_cols = ['puuid', 'matchid', \
              'win', 'kills', 'deaths', 'assists', 'bountyGold', 'turretKills', 'turretsLost', \
              'totalDamageDone15', 'totalDamageDoneToChampions15', 'totalDamageTaken15', 'level15', 'xp15', 'minionsKilled15', 'totalGold15', \
              'kills15', 'deaths15', 'assists15']

In [45]:
df_match = pd.DataFrame(columns=match_cols)
df_match

,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15


In [152]:
def make_base_dict(puuid, matchid):
    base_dict = {'puuid': puuid, 'matchid': matchid}
    
    return base_dict

In [138]:
def response_match(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid
    resp_ma = requests.get(url, headers=request_header)

    return resp_ma

In [139]:
# participant ID 찾기
### match 에서는 pid-1
### timeline 에서는 'pid'
def find_participantId(resp_ma, puuid):
    return resp_ma.json()['metadata']['participants'].index(puuid) + 1

In [140]:
def make_data_end_dict(resp_ma, parti_id):
    data_end = resp_ma.json()['info']['participants'][parti_id - 1]

    data_end_dict = {'win': data_end['win'],
                     'kills': data_end['kills'],
                     'deaths': data_end['deaths'],
                     'assists': data_end['assists'],
                     'bountyGold': data_end['challenges']['bountyGold'],
                     'turretKills': data_end['turretKills'],
                     'turretsLost': data_end['turretsLost']
                    }
    
    return data_end_dict

In [130]:
def response_timelines(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid + '/timeline'
    resp_tl = requests.get(url, headers=request_header)
    
    return resp_tl

In [131]:
def make_data_15_dict(resp_tl, parti_id):
    data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(parti_id)]

    data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                    'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                    'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                    'level15': data_15['level'],
                    'xp15': data_15['xp'],
                    'minionsKilled15': data_15['minionsKilled'],
                    'totalGold15': data_15['totalGold']
                    }
    
    return data_15_dict

In [132]:
def make_data_15_kda_dict(resp_tl, parti_id):
    data_15_kda_dict = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

    for times in range(0, 16):
        events = resp_tl.json()['info']['frames'][times]['events']

        for i in range(len(events)):
            if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
                
                if parti_id == events[i]['killerId']:
                    data_15_kda_dict['kills15'] += 1

                if parti_id == events[i]['victimId']:
                    data_15_kda_dict['deaths15'] += 1

                if parti_id in events[i]['assistingParticipantIds']:
                    data_15_kda_dict['assists15'] += 1

    return data_15_kda_dict

In [ ]:
for i in range(len(df2)):
    puuid = df2.loc[[i]]['puuid']
    match_ids = list(df2.loc[0])[3:]
    
    for matchid in match_ids:
        base_dict = make_base_dict(puuid, matchid)
        
        resp_ma = response_match(matchid)
        parti_id = find_participantId(resp_ma, puuid)
        data_end_dict = make_data_end_dict(resp_ma, parti_id)

        resp_tl = response_timelines(matchid)
        data_15_dict = make_data_15_dict(resp_tl, parti_id)
        data_15_kda_dict = make_data_15_kda_dict(resp_tl, parti_id)

        base_dict.update(data_end_dict)
        base_dict.update(data_15_dict)
        base_dict.update(data_15_kda_dict)
        df_match.append(base_dict, ignore_index=True)

        time.sleep(3)
        

#### 연습구간

In [8]:
# parti_ID 찾기
def find_participantId(response, puuid):
    return response.json()['metadata']['participants'].index(puuid) + 1

In [11]:
find_participantId(resp_tl, df2['puuid'][0])

9

In [ ]:
resp_tl.json()['info']['frames'][15]['participantFrames']['9']

In [38]:
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDone']) # 15분까지 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDoneToChampions']) # 15분까지 챔피언에게 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageTaken']) # 15분까지 받은 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['level']) # 15분 레벨
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['minionsKilled']) # 15분 미니언 처치 수
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['totalGold']) # 15분 골드 누적 획득량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['xp']) # 15분 경험치 누적 획득량

# 게임 종료
print(response.json()['info']['participants'][8]['challenges']['bountyGold']) # 현상금
print(response.json()['info']['participants'][8]['kills']) # 킬
print(response.json()['info']['participants'][8]['deaths']) # 데스
print(response.json()['info']['participants'][8]['assists']) # 어시스트
print(response.json()['info']['participants'][8]['turretKills']) # 자신이 파괴한 포탑 수
print(response.json()['info']['participants'][8]['turretsLost']) # 전체 파괴된 포탑 수
print(response.json()['info']['participants'][8]['win']) # 승패 여부

55723
4047
2377
9
141
6647
5512
650
5
2
3
False
2
9
False


In [25]:
resp_tl.json()['info']['frames'][8]['events'][22].keys()

dict_keys(['assistingParticipantIds', 'bounty', 'killStreakLength', 'killerId', 'position', 'shutdownBounty', 'timestamp', 'type', 'victimDamageDealt', 'victimDamageReceived', 'victimId'])

In [34]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

6 [3, 4, 5] 2 7
8 [2, 4, 5] 3 10
22 [2, 5] 4 9
	death


##### 테스트

In [80]:
pud = df2['puuid'][0]
mid = df2['match1'][0]
basedict = {'puuid': pud, 'matchid': mid}

In [51]:
url1 = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid
resp_end = requests.get(url1, headers=request_header)
part_id = find_participantId(resp_end, pud)

In [52]:
part_id

9

In [82]:
data_end = resp_end.json()['info']['participants'][part_id-1]
data_end_dict = {'win': data_end['win'],
                 'kills': data_end['kills'],
                 'deaths': data_end['deaths'],
                 'assists': data_end['assists'],
                 'bountyGold': data_end['challenges']['bountyGold'],
                 'turretKills': data_end['turretKills'],
                 'turretsLost': data_end['turretsLost']
                 }

In [117]:
data_end_dict

{'win': False,
 'kills': 5,
 'deaths': 2,
 'assists': 3,
 'bountyGold': 650,
 'turretKills': 2,
 'turretsLost': 9}

In [86]:
url_tl = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid + '/timeline'
resp_tl = requests.get(url_tl, headers=request_header)

In [89]:
data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(part_id)]
data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                'level15': data_15['level'],
                'xp15': data_15['xp'],
                'minionsKilled15': data_15['minionsKilled'],
                'totalGold15': data_15['totalGold']
                }

In [118]:
data_15_dict

{'totalDamageDone15': 55723,
 'totalDamageDoneToChampions15': 4047,
 'totalDamageTaken15': 2377,
 'level15': 9,
 'xp15': 5512,
 'minionsKilled15': 141,
 'totalGold15': 6647}

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

In [114]:
data_15_kda = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

In [115]:
for times in range(0, 16):
    events = resp_tl.json()['info']['frames'][times]['events']

    for i in range(len(events)):
        if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
            
            if part_id == events[i]['killerId']:
                data_15_kda['kills15'] += 1

            if part_id == events[i]['victimId']:
                data_15_kda['deaths15'] += 1

            if part_id in events[i]['assistingParticipantIds']:
                data_15_kda['assists15'] += 1

In [116]:
data_15_kda

{'kills15': 2, 'deaths15': 1, 'assists15': 2}

In [119]:
basedict.update(data_end_dict)
basedict.update(data_15_dict)
basedict.update(data_15_kda)

In [120]:
basedict

{'puuid': 'ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEWXvHWwNMzslgzzwziCo-NkdTRnTYOE_5w',
 'matchid': 'KR_6626090481',
 'win': False,
 'kills': 5,
 'deaths': 2,
 'assists': 3,
 'bountyGold': 650,
 'turretKills': 2,
 'turretsLost': 9,
 'totalDamageDone15': 55723,
 'totalDamageDoneToChampions15': 4047,
 'totalDamageTaken15': 2377,
 'level15': 9,
 'xp15': 5512,
 'minionsKilled15': 141,
 'totalGold15': 6647,
 'kills15': 2,
 'deaths15': 1,
 'assists15': 2}

In [122]:
df_match.append(basedict, ignore_index=True)

,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15
0,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6626090481,False,5,2,3,650,2,9,55723,4047,2377,9,5512,141,6647,2,1,2


##### -----

In [61]:
tempdf = df_match.copy()
tempdf


,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15


In [74]:
temp_dict = {'win':True, 'kills':10, 'deaths':20, 'assists':30}
td2 = {'new': False}

In [75]:
tempdf = pd.DataFrame(columns=['win', 'kills', 'deaths', 'assists', 'new'])

In [71]:
for key, value in temp_dict.items():
    print(key, value)
    tempdf.append(temp_dict, ignore_index=True)

tempdf

win True
kills 10
deaths 20
assists 30


,win,kills,deaths,assists


In [77]:
temp_dict.update(td2)

In [78]:
temp_dict

{'win': True, 'kills': 10, 'deaths': 20, 'assists': 30, 'new': False}

In [79]:
tempdf.append(temp_dict, ignore_index=True)

,win,kills,deaths,assists,new
0,True,10,20,30,False


In [43]:
requests.get('http://ddragon.leagueoflegends.com/cdn/13.17.1/data/en_US/item.json').json()['data']['1001']['image']['full']

'1001.png'

#### 연습구간 종료

In [107]:
url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + df2['match1'][0] + '/timeline'
response = requests.get(url, headers=request_header)
printjs(response.json())

{
  "metadata": {
    "dataVersion": "2",
    "matchId": "KR_6626090481",
    "participants": [
      "4tpl29Z5nxL6zitkO2fantUyNZzLTPXaE7Yr-3NOjkWbVb_CbFx_yybToR8-YAamh3VmYyOfrunDQA",
      "BCZNQ1p2YyWAHofUgSMetluhlihOuE43yYY9KbrZgc8NF1Hi18p8LnYIEN3mILiJw1NSOIfzBqeyNg",
      "sc4BKSqK0l-Txan5IBrE0Dylcu2ocoA55zq8TQ3NulcqFWDdMlz7ZAUkv2BjG8qKqja5h2ObHpCDYw",
      "jlILAEZm9tdrVs00vuXIIyAhyrPKmC4f3xr-WBUcYFMC0um3ImcR8mG5Hqu0i5myAFBTJclOmPK_yg",
      "fqlEIZlWLOKeglpZuOgYd1MLZi-P1BGDNsMSDJWQWIQz7eOkNtc57Z-JFytSg9x8U58quQ-gOxVtsA",
      "VXJthY5LTAbnWxEI31YVIUCHfF9H-2vLrOeE9rGYhRJRrPEZKkefUxwX2CazFcilq0XtF1--uOR8BA",
      "lNivXCjM8Ffbm1hOeHC50Pt152ufciPfiAu3s8a983IUZMxJAGUqp41BxuR02GboYQCAduxRN__5sg",
      "cP0q5MSCzU-ROGzRxsuLVgLWkKYmQHsxbyE-BORH4PUr9XLoNYeGTF9VzwnFgLjA6xyrCquXA8pV6w",
      "ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEWXvHWwNMzslgzzwziCo-NkdTRnTYOE_5w",
      "x28Hm0cWRA2uugv5nYx1uenzH59MTDKLNuqzc421PhtRYPrDS0hgWaxCTdOACfs5RDxAat5B4_DylA"
    ]
  },
  "info": {
  

In [109]:
response.json()['info'].keys()

dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

In [111]:
response.json()['info']['frames']

[{'events': [{'realTimestamp': 1690813674397,
    'timestamp': 0,
    'type': 'PAUSE_END'}],
  'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
     'abilityPower': 0,
     'armor': 36,
     'armorPen': 0,
     'armorPenPercent': 0,
     'attackDamage': 25,
     'attackSpeed': 100,
     'bonusArmorPenPercent': 0,
     'bonusMagicPenPercent': 0,
     'ccReduction': 0,
     'cooldownReduction': 0,
     'health': 625,
     'healthMax': 625,
     'healthRegen': 0,
     'lifesteal': 0,
     'magicPen': 0,
     'magicPenPercent': 0,
     'magicResist': 28,
     'movementSpeed': 345,
     'omnivamp': 0,
     'physicalVamp': 0,
     'power': 0,
     'powerMax': 150,
     'powerRegen': 0,
     'spellVamp': 0},
    'currentGold': 500,
    'damageStats': {'magicDamageDone': 0,
     'magicDamageDoneToChampions': 0,
     'magicDamageTaken': 0,
     'physicalDamageDone': 0,
     'physicalDamageDoneToChampions': 0,
     'physicalDamageTaken': 0,
     'totalDamageDone': 0,
     'totalD

In [112]:
temp = response.json()['info']
len(temp['frames'])

28

In [113]:
for x in temp['frames']:
    del x['participantFrames']

In [115]:
temp['frames']

[{'events': [{'realTimestamp': 1690813674397,
    'timestamp': 0,
    'type': 'PAUSE_END'}],
  'timestamp': 0},
 {'events': [{'itemId': 1056,
    'participantId': 1,
    'timestamp': 2020,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 3340,
    'participantId': 1,
    'timestamp': 2256,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 2003,
    'participantId': 1,
    'timestamp': 2489,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 2003,
    'participantId': 1,
    'timestamp': 2623,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 3858,
    'participantId': 10,
    'timestamp': 4094,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 2003,
    'participantId': 10,
    'timestamp': 4094,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 2003,
    'participantId': 10,
    'timestamp': 4094,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 3340,
    'participantId': 10,
    'timestamp': 4426,
    'type': 'ITEM_PURCHASED'},
   {'itemId': 1101,
    'participantId': 2,
    'timestamp': 5964,
    'type': 'ITEM_PURCH

#### 선택
KDA, bountyLevel, champLevel, goldEarned, timePlayed, totalDamageDealt, totalDamageTaken, win

#### 15분
* 서렌 가능 시간
* 포탑 방패가 사라지는 14분 + 이후 타워가 깨질 수 있는 시간 약 1분
* 챔피언 성능이 달라지는 1코어 이상(1.5코어) 아이템 소지

#### 10분?
* 다른 라이너의 큰 개입 없이 라인전이 이루어지는 시간
* 서포터까지도 충분히 궁극기를 배우고 교전에 적극 참여하기 충분한 시간
* 12~13분이면 라인전이 끝남, 15분은 거의 3번째 용